In [18]:
column_string = ', '.join(map(lambda x: x + ' TEXT', tableSchema))
'CREATE TABLE IF NOT EXISTS {0} ({1})'.format(tableName, column_string)

'CREATE TABLE IF NOT EXISTS tweetTable (created_at TEXT, geo TEXT, id_str TEXT, place TEXT, retweet_count TEXT, favorite_count TEXT, source TEXT, text TEXT, user_description TEXT, user_followers_count TEXT, user_friends_count TEXT, user_id_str TEXT, user_location TEXT, user_name TEXT, user_time_zone TEXT)'

In [14]:
dbName = 'test.db'
tableName = 'tweetTable'
tableSchema = ['created_at',
               'geo',
               'id_str',
               'place',
               'retweet_count',
               'favorite_count',
               'source',
               'text',
               'user_description',
               'user_followers_count',
               'user_friends_count',
               'user_id_str',
               'user_location',
               'user_name',
               'user_time_zone']
# tableSchema = {'created_at': 'TEXT',
#                'geo': 'TEXT',
#                'id_str': 'TEXT',
#                'place': 'TEXT',
#                'retweet_count': 'INTEGER',
#                'favorite_count': 'INTEGER',
#                'source': 'TEXT',
#                'text': 'TEXT',
#                'user_description': 'TEXT',
#                'user_followers_count': 'INTEGER',
#                'user_friends_count': 'INTEGER',
#                'user_id_str': 'TEXT',
#                'user_location': 'TEXT',
#                'user_name': 'TEXT',
#                'user_time_zone': 'TEXT'}

In [ ]:
import sqlite3


class MyStreamWriter(object):

    def __init__(self, dbName, tableName, tableSchema):
#         if not isinstance(tableSchema, dict):
#             raise ValueError('tableSchema type must be Python dict')
        self.tableName = tableName
        self.tableSchema = tableSchema
        self.insert_values = []
        ## Create database connection
        self.con = sqlite3.connect(dbName)
        self.cur = self.con.cursor()
        ## Construct create table query
        column_string = ', '.join(map(lambda x: x + ' TEXT', tableSchema))
        create_table = 'CREATE TABLE IF NOT EXISTS {0} ({1})'.format(self.tableName, 
                                                                     column_string)
        ## Create table
        self.cur.execute(create_table)
        self.con.commit()

    def stream_in(self, tableValue):
#         if not isinstance(tableValue, dict):
#             raise ValueError('tableValue type must be Python dict')
        ## Construct insert value query
        value_string = '("' + '","'.join(tableValue) + '")'
       
        self.insert_values.append(value_string)
        ## Insert into table
        if len(self.insert_values) > 99:
            insert_values = 'INSERT INTO {0} VALUES {1};'.format(self.tableName,
                                                                 ', '.join(self.insert_values))
            self.cur.execute(''.join(insert_values))
            self.con.commit()
            self.insert_values = []

    def read_table(self):
        self.cur.execute('SELECT * FROM {0}'.format(self.tableName))
        return [tableSchema.keys()] + self.cur.fetchall()

    def disconnect(self):
        self.con.close()


In [10]:
import tweepy


class MyStreamListener(tweepy.StreamListener):

    def on_connect(self):
        self.writer = MyStreamWriter(dbName, tableName, tableSchema)

    def on_status(self, status):
        ## Create tableValue
        tableValue = list()
        for key in tableSchema:
            if key.find('user_') == 0:
                k1, k2 = key.split('_', 1)
                tableValue[key] = unicode(status.__dict__[k1].__dict__[k2]).encode('ascii', 'ignore').replace('"', '').strip()
            else:
                tableValue[key] = unicode(status.__dict__[key]).encode('ascii', 'ignore').replace('"', '').strip()
        ## Insert values into table
        print(tableValue)
        self.writer.stream_in(tableValue)

    def on_error(self, status_code):
        if status_code == 420:
            print('error code: 420')
            self.db.disconnect()
            # returning False in on_data disconnects the stream
            return False

In [11]:
consumer_token = '0ec7PKFpXR57cCEFWYarQkToE'
consumer_secret = 'yeDFsy7Wpaadxp2eoUWMlrlGWDGItzNRk0nKhBJtVZ29enxxzf'
access_token = '2604803370-v8QJYcbcuJGs0ugZHnUXM7bcYygutgdxawQn3ZL'
access_token_secret = 'aA04xPHJX1QBdqMMUSn9Ji1c6fubTXbOB2ojqz6rlLVis'
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
with open('FoodList_cleaned.txt', 'rb') as f:
    food_list = [line.strip() for line in f]

In [ ]:
myStream = tweepy.Stream(auth=auth, listener=MyStreamListener())

In [ ]:
myStream.filter(track=food_list, languages=['en']) #, filter_level='medium') #, async=False)

In [ ]:
myStream.disconnect()

In [ ]:
api = tweepy.API(auth_handler=auth)

In [12]:
from twython import Twython

twitter = Twython(consumer_token, consumer_secret, access_token, access_token_secret)

In [13]:
tweet = twitter.show_status(id=805468578745122817)
tweet

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Sun Dec 04 17:47:28 +0000 2016',
 u'entities': {u'hashtags': [],
  u'symbols': [],
  u'urls': [],
  u'user_mentions': []},
 u'favorite_count': 3,
 u'favorited': False,
 u'geo': None,
 u'id': 805468578745122817,
 u'id_str': u'805468578745122817',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'is_quote_status': False,
 u'lang': u'en',
 u'place': None,
 u'retweet_count': 0,
 u'retweeted': False,
 u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 u'text': u'"I gave up getting laid last night to eat a burrito!?!!"- my lovely teammate',
 u'truncated': False,
 u'user': {u'contributors_enabled': False,
  u'created_at': u'Mon May 25 20:35:28 +0000 2009',
  u'default_profile': False,
  u'default_profile_image': False,
  u'description': u"Potatoes,banana suits

In [ ]:
u = api.get_user(42481592)
print u.screen_name

In [ ]:
'CREATE TABLE {0} IF NOT EXIST ({1})'.format('mytable')

In [ ]:
tableSchema = {'a':'INTEGER', 'b':'TEXT'}
tableValue = {'a':'3', 'b':'txt'}

In [ ]:
', '.join(map(lambda x: ' '.join(x), tableSchema.items()))

In [3]:
len(filter(lambda x: x != (u'None',), geos))

15515

(u"{u'type': u'Point', u'coordinates': [40.78712093, -111.98129839]}",)

In [6]:
import json

In [21]:
u"{u'type': u'Point', u'coordinates': [40.78712093, -111.98129839]}".replace("'", "\"")

u'{u"type": u"Point", u"coordinates": [40.78712093, -111.98129839]}'

In [33]:
json.loads(u'{u"type": u"Point", u"coordinates": [40.78712093, -111.98129839]}'.replace('u"', '"'))

{u'coordinates': [40.78712093, -111.98129839], u'type': u'Point'}

In [32]:
json.loads(u'{u"bar":[u"baz", null, 1.0, 2]}'.replace('u"', '"'))

{u'bar': [u'baz', None, 1.0, 2]}